In [17]:
# -*- coding:utf-8 -*-
# library is required for the test
# pip install --upgrade google-api-python-client 
from googleapiclient.discovery import build
import pprint
import time, re, requests, os
from bs4 import BeautifulSoup
import urllib.parse

In [18]:
# You have to take a key from google CSE
# You can get an API key from https://developers.google.com/maps/documentation/javascript/get-api-key?hl=ko#key
my_api_key = "** YOUR API KEY **"

# You can setup the CSE and get CSE_ID from https://cse.google.com/cse/all
# You can add searching site in GUI interface 
# but it seems like *only one site per key is working properly* at the moment (17.06.05)
my_cse_id= "** YOUR CSE ID **"

# Insert the site in the CSE setup
search_site = "** SITE in the CSE setup **"

In [19]:
# do Search with Saving
def doSearchWithSaving(search_term):
    search_term.replace(" ", "+")
    NextPage=0
    cur = 0
    while NextPage!=-1:
        #print("CUR : " ,cur)
        linkSet, NextPage, NumOfResult = getGoogleSearchLink(search_term, search_site, NextPage)
        for idx, link_url in enumerate(linkSet):
            filename='{}/[{}of{}]_{}.txt'.format(search_site, cur+1, NumOfResult, search_term.replace(" ","_"))
            SaveLinkToFile(filename, link_url)
            cur += 1
        time.sleep(1)
        if cur>100:
            print("Finish")
            break

# Google Search Func
def google_search_init(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

# Get Google Search Links of the website
def getGoogleSearchLink(search_term, search_site, cur_page):
    #start=res['queries']['nextPage'][0]['startIndex'],
    queryStr = '{} site:{}'.format(search_term, search_site)
    if cur_page==0:
        results = google_search_init(queryStr, my_api_key, my_cse_id, num=10)
    else:
        results = google_search_init(queryStr, my_api_key, my_cse_id, num=10, start=cur_page)
    links = [k['link'] for k in results['items']]
    NumOfResult = results['queries']['request'][0]['totalResults']
    if results['queries'].get('nextPage'):
        NextPage=results['queries']['nextPage'][0]['startIndex']
    else:
        NextPage=-1
    return links, NextPage, int(NumOfResult)

# Save URL to FILE
def SaveLinkToFile(filename, link_url):
    r = requests.get(link_url)
    soup = BeautifulSoup(r.content, 'html.parser') #  encoding='utf-8'
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    visible_text = soup.getText()
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    with open(filename, 'wt', encoding='utf-8') as fwrite:
        fwrite.write(visible_text)
        fwrite.close()

In [16]:
# Test Code 
# Google CSE provides 10 pages of result and 100 query per day
doSearchWithSaving("송도 파스타")